In [ ]:
import pandas as pd
import yfinance as yf
import requests as req

In [ ]:
# Indicadores que queremos encontrar
# VPA - Valor Patrimonial por Ação - 1.5x
# LPA - Lucro por Ação - 15x
# (22.5 * VPA * LPA) ** (1/2)